In [22]:
import urllib.request
import requests
from bs4 import BeautifulSoup
import pandas as pd
import urlTools
import time 


# In[2]:
#location="`D:\WHIN\"

def get_base_url(url):
    from urllib.parse import urlsplit
    base_url = "{0.scheme}://{0.netloc}/".format(urlsplit(url))
    

    if base_url.startswith("https"):
        base_url = base_url.replace('https://','')
    elif base_url.startswith("http"):
        base_url = base_url.replace('http://','')
        
    if base_url.endswith('/'):
        base_url = base_url.replace('/','')
    return base_url


# In[3]:


def does_page_exist(url):
    exists = 0 
    try:
        request = requests.get(url)
        if request.status_code == 200:
            exists = 1 
            
    except: 
        pass
    
    return exists


# In[4]:


# Function to extract links from a URL------------------------------------------
def href_scrapper(url):

    try:
        page = requests.get(url)
        soup = BeautifulSoup(page.content, 'html.parser')
        a_tag = soup.find_all('a')
        img_tag = soup.find_all('img')
        link_tag = soup.find_all('link')
        link_list = list()

        for links in a_tag:
            link_list.append(links.get('href'))

        for links in link_tag:
            link_list.append(links.get('href'))

        for links in img_tag:
            link_list.append(links.get('src'))

        return link_list

    except:
        return 0



# In[5]:


# Function to create complete links---------------------------------------------
def link_former(url):
    x = url

    if x.startswith("http"):
        return x

    else:
        full_url = original_url + x
        return full_url



# In[6]:


# Function to save links to directory of python file----------------------------
def file_save(url, i):
    save_error = 0
    full_url = url
    try:
        if str(full_url).endswith(".htm"):
            name = "WebPage" + str(i) + ".htm"
            urllib.request.urlretrieve(full_url, name)
            m = i + 1

        elif str(full_url).endswith(".pdf"):
            name = "Pdf" + str(i) + ".pdf"
            urllib.request.urlretrieve(full_url, name)
            m = i + 1

        elif str(full_url).endswith(".jpg"):
            name = "Image" + str(i) + ".jpg"
            urllib.request.urlretrieve(full_url, name)
            m = i + 1

        elif str(full_url).endswith(".jpeg"):
            name = "Image" + str(i) + ".jpeg"
            urllib.request.urlretrieve(full_url, name)
            m = i + 1

        elif str(full_url).endswith(".png"):
            name = "Image" + str(i) + ".png"
            urllib.request.urlretrieve(full_url, name)
            m = i + 1

        elif str(full_url).endswith(".css"):
            name = "CSS File" + str(i) + ".css"
            urllib.request.urlretrieve(full_url, name)
            m = i + 1

        else:
            name = "WebPage" + str(i) + ".html"
            urllib.request.urlretrieve(full_url, name)
            m = i + 1

        return m
    except:
        save_error = save_error + 1




# In[7]:

def crawl(original_url, num_Id):
    
    # Accepting input of URL and depth----------------------------------------------
    tic = time.time()
    
    parent_list = [original_url]
    url_to_check = get_base_url(original_url)
    print(url_to_check)
    url_to_check = str(url_to_check)

    layer_stop = 1
    layer = 0


    #initializing all the requiered lists---------------------------------------------
    visited_all  = [original_url]
    visited_current_layer = []
    child_list =[]
    child_list_filtered = []
    #columns = ['Link','Parent Link', 'Layer']
    df = pd.DataFrame()


    # Main execution of scrapper----------------------------------------------------

    #looping through layers 
    while layer < layer_stop and layer <7:
    
        #looping through URLs in parent-list
        for url in parent_list:
        
            #scraping the children from the parent url----------------------------
            if href_scrapper(url) != 0:
                child_list = href_scrapper(url)
    
    
            for child in child_list:
                if child != None:
                    #if child link is of the form "index.php/blahblah" and parent ends with '/'
                    #---> "parentlink/index.php/blahblah"
                    if child.startswith('/'):
                        child= str(url) + str(child)
                    
                    if url.endswith('/') and url_to_check not in child:
                        child = str(url) + str(child)
                    
                #normalize the child links-------------------------------------
                    child=urltools.normalize(child)  
                    

                    #filtering out based on 1) External 2) Repeating 3) Invalid links---------------------------
                    if url_to_check in child and child not in visited_all and does_page_exist(child)==1:
                        child_list_filtered.append(child)
                
                    #adding everthing to visited all--------------------
                    if child not in visited_all:
                        child_slash = child + '/'
                        visited_all.append(child)  
                        visited_all.append(child_slash)
                    
                    
            #adding  the visited and filtered children into the "current visited layer" ------ 
            for child_filtered in child_list_filtered:
                visited_current_layer.append(child_filtered)

            #creating a Pandas dataframe to store everything for download----------
            layer_number = [layer+1]*len(child_list_filtered)
            parent_of_child = [url]*len(child_list_filtered)
            for visited_current in visited_current_layer: 
                print(visited_current)
            if(not visited_current.endswith('.png') and not visited_current.endswith('.jpg') and not  visited_current.endswith('.pdf')):
                parent_list.append(visited_current)

            df_child = pd.DataFrame(child_list_filtered)
            df_parent = pd.DataFrame(parent_of_child)
            df_layer = pd.DataFrame(layer_number)


            df_to_be_added = pd.concat([df_child,df_parent,df_layer], axis=1)
            df = pd.concat([df,df_to_be_added],ignore_index=True, axis = 0)
            #----------------------------------------------------------------------
        
            #emptying the child lists
            child_list = []
            child_list_filtered = []
        
        #condition to stop filtering-----------------------------------------------
        if not visited_current_layer :
            layer_stop = layer_stop 
        else:
            layer_stop += 1

    
        #child layer is now parent layer--------------------------------------------
        parent_list = []
    
        #we only dont add .png, .jpg , .pdf to the new parent layer 
        
            
            
        #displaying the links in different layers----------------------------------
        #print("Links in LAYER:" + str(layer+1))
        print("No of links = " + str(len(visited_current_layer)))
        #print(visited_current_layer)
        print("\n")
        visited_current_layer = [] 
        #updating the layer number
        layer +=1
    file_name = 'D:\WHIN\website'
    df.to_csv(file_name + str(num_Id) + '.csv', sep=',', encoding='utf-8')
    
    return df, num_Id


ModuleNotFoundError: No module named 'urlTools'

In [12]:
import requests

page = requests.get("https://docs.python.org/3/howto/urllib2.html")
from bs4 import BeautifulSoup
from bs4.element import Comment
soup = BeautifulSoup(page.content, 'html.parser')


#information based on:
def tag_visible(element):
    if element.parent.name in ['style', 'script', 'head', 'title', 'meta', '[document]']:
        return False
    if isinstance(element, Comment):
        return False
    return True
p_tag=soup.find_all('p')

#extract text from html: 
def text_from_html(body):
    soup = BeautifulSoup(body, 'html.parser')
    texts = soup.findAll(text=True)
    visible_texts = filter(tag_visible, texts)  
    return u" ".join(t.strip() for t in visible_texts)
web_text = []


In [13]:
import json
import urllib
import os 
os.chdir("D:\WHIN/")
def scrape(df, num_Id):
    TempD={}
    for index,item in df.iterrows():
        try:      
            print(item[1])
            html = urllib.request.urlopen(str(item[1])).read()
            web_text.append(text_from_html(html))
            print(text_from_html(html))
            save_url(str(item[1]), index)
            TempD[item[1]]=text_from_html(html)
        except: 
            pass
    with open('json'+str(num_Id)+'.json', 'w') as outfile:  
        json.dump(TempD, outfile)
        
    return 1      

In [14]:
import pandas as pd
from pandas import ExcelWriter
from pandas import ExcelFile
excel_file = 'D:\WHIN/Trial1.xlsx'
df = pd.read_excel(excel_file , sheet_name='From List of Companies')
df


,Serial No.,Company Name,Website,"Has a Website? (1=Yes, 0=No)",Static/Dynamic Website?,"If no website- Is present on Facebook? (1=Yes, 0=No)"
0,2.0,Carmel Engineering,http://www.carmeleng.com/,1.0,D,NaN
1,3.0,Dayton-Phoenix Group,https://www.dayton-phoenix.com/,1.0,D,NaN
2,4.0,Dyna-Fab,http://dyna-fab.org/,NaN,NaN,NaN
3,5.0,Oerlikon Fairfield,https://www.oerlikon.com/fairfield/en/,NaN,NaN,NaN
4,6.0,Grand Industrial,grandindustrial.com,NaN,NaN,NaN
5,7.0,Harrison Steel Castings,http://www.hscast.com/,NaN,NaN,NaN
6,8.0,Hose Technology,http://hosetec.com/,NaN,NaN,NaN
7,9.0,Kirby Risk Service Center,https://www.kirbyrisk.com,NaN,NaN,NaN
8,10.0,Lafayette Wire Products,http://lafayettewire.com/,NaN,NaN,NaN
9,11.0,Logan Stampings,http://loganstampings.com/,NaN,NaN,NaN


In [17]:
#this is the code to run everything at once: 
i = 1 
for index,row in df.iterrows():
    num_Id=row[0]
    original_url=row[2]
    try:
        websites= pd.read_csv("website"+str(row[0])+ '.csv')
    except:
        websites, num_Id = crawl(row[2],row[0])
        filename = "website" + str(int(row[0]) )+ ".csv"
        df_of_child_links = pd.read_csv(filename)
        df_of_child_links
        flag = scrape(df_of_child_links ,i) 
        
    i = i + 1 

www.carmeleng.com


NameError: name 'urltools' is not defined

In [4]:
import json
from pprint import pprint

with open('json1.json') as f1:
    data1 = json.load(f1)
    
with open('json2.json') as f2:
    data2 = json.load(f2)
    

